In [0]:
!pip install tensorflow-gpu==2.0.0-beta1

     |████████████████████████████████| 348.9MB 101kB/s 
     |████████████████████████████████| 501kB 33.4MB/s 
     |████████████████████████████████| 3.1MB 32.3MB/s 


In [0]:
from google.colab import drive
drive.mount("/content/drive")
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


Mounting from G Drive - but can be used locally as well

In [0]:
# zip file directory
train_file = "/content/drive/My Drive/Colab Notebooks/"

Prepping zip file for extraction

In [0]:
import zipfile

with zipfile.ZipFile(train_file, 'r') as y:
  y.extractall()

In [0]:
!ls

drive  sample_data  training_set


This is an if else of what the item we're training for is.
It checks the label and matches it up with the image

In [0]:
import os
import cv2

def load_image(file_path):
  return cv2.imread(file_path)

def extract_label(file_name):
  return 1 if "item" in file_name else 0
#folder directory, not .zip
train_path = "./training_set/"
image_files = os.listdir(train_path)
train_images = [load_image(train_path + file) for file in image_files]
train_labels = [extract_label(file) for file in image_files]

In [0]:
!ls

drive  sample_data  training_set


In [0]:
def preprocess_image(img, side=96):
    min_side = min(img.shape[0], img.shape[1])
    img = img[:min_side, :min_side]
    img = cv2.resize(img, (side,side))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img / 255.0

Check and make sure directory reads into dogs/ folder. Issue previously was to do with reading into another zip file

Preprocess, plot and resize the image. 
preview_index is just for checking

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
preview_index = 1
plt.subplot(1, 2, 1)
plt.imshow(train_images[preview_index])
plt.subplot(1, 2, 2)
plt.imshow(preprocess_image(train_images[preview_index]), cmap="gray")

Convert all imagery into grayscale before punching it through training. This doesn't always have to be the case, it's just more efficient

Funneling through individually to control RAM usage

In [0]:
for i in range(len(train_images)):
    train_images[i] = preprocess_image(train_images[i])

In [0]:
import numpy as np
  
train_images = np.expand_dims(train_images, axis = -1)
train_labels = np.array(train_labels)
print(train_images.shape, train_labels.shape)

Filter content in increments of x *= 2 which is x = x * 2

In [0]:
import tensorflow as tf

print("Tensorflow version is: ", tf.__version__)
filter_quantity = 16
layers = [
          tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding="same", activation= tf.nn.relu, input_shape = train_images.shape[1:]),
          tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
          tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation= tf.nn.relu),
          tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
          tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", activation= tf.nn.relu),
          tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
          tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation= tf.nn.relu),
          tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
          tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation= tf.nn.relu),
          tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),

          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(units= 512, activation= tf.nn.relu),
          tf.keras.layers.Dense(units= 256, activation= tf.nn.relu),
          tf.keras.layers.Dense(units= 2, activation= tf.nn.softmax)
]

model = tf.keras.Sequential(layers)
model.compile(optimizer=tf.optimizers.Adam(),
              #because they're only numbers, we got with SCC
              loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])

Tensorflow version is:  2.0.0-beta1


AttributeError: ignored

Train the Model and save the weights

In [0]:
model.fit(train_images, train_labels, epochs=5, batch_size=50)
model.save_weights("koalaModel.tf")

NameError: ignored

Upload files that you're wanting to test

In [0]:
from google.colab import files
uploads = files.upload()

Creating a new model with previous layers. Load the weights from previously. Get the predictions probabilities and display the images for testing

In [0]:
eval_images = [preprocess_image(load_image(file)) for file in uploads.keys()]
eval_model = tf.keras.Sequential(layers)
eval_model.load_weights("koalaModel.tf")
eval_predictions = eval_model.predict(np.expand_dims(eval_images, axis= -1)) #will be deprecated in the future

cols = 4
rows = np.ceil(len(eval_images)/cols) # remember this formula
fig = plt.gcf()
fig.set_size_inches(cols * 4, rows * 4)
for i in range(len(eval_images)):
    plt.subplt(rows, cols, i + 1)
    plt.imshow(eval_images[i], cmap="gray")
    plt.title("IdentifiedItem" if np.argmax(eval_predictions[i]) == 1 else "NotKeyIdentifiedItem")
    plt.axis('off')


In [0]:
!ls

Grab the saved weights from the files at runtime

In [26]:
files.download("koalamodel.tf.index")

FileNotFoundError: ignored